# Image Processing SS 20 - Assignment 06

### Deadline is 03.06.2020 at 11:55 am

Please solve the assignments together with a partner.
I will run every notebook. Make sure the code runs through. Select `Kernel` -> `Restart & Run All` to test it.
Please strip the output from the cells, either select `Cell` -> `All Output` -> `Clear` or use the `nb_strip_output.py` script / git hook.

In [ ]:
# display the plots inside the notebook
%matplotlib inline

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pylab
import scipy.io.wavfile
from io import BytesIO
import urllib.request as urllib2
import ssl
from math import e,pi,sqrt
import IPython.display as adt
import scipy.signal as sp 
from scipy.linalg import dft
ssl._create_default_https_context = ssl._create_unverified_context
pylab.rcParams['figure.figsize'] = (23, 5)   # This makes the plot bigger

# Exercise 1 - Crosscorrelation - 2 Points
Implement a function myCrossCorrelation, which takes two discrete functions in the form of arrays as an input and outputs the resulting crosscorrelation function. Now apply your correlation function as follows:
a) use as input function 1 an array of 10 random numbers (0-255), function 2 shall be [1], plot the two functions and the result of the crosscorrelation of both.
b) use the same function 1 as in a), function b) shall be [1 2 1]. Plot the two functions and the crosscorrelation of both.
c) repeat the first two tasks but swap the arguments

In [ ]:
np.random.seed(500)

In [ ]:
def myCrossCorrelation(func1, func2):
    result = np.zeros_like(func1)
    func2 = np.flip(func2)
    for i,val in enumerate(func1):
        c = func2[ :i + 1]
        length = len(c)
        for j in range(length):
            result[i] += c[length - j  - 1 ] * func1[i - j]
    return result

random = np.random.randint(low=0,high=255,size=10)
a = myCrossCorrelation(random, np.array([1]))
b = myCrossCorrelation(random, np.array([1,2,1]))
c1 = myCrossCorrelation(np.array([1]),random )
c2 = myCrossCorrelation( np.array([1,2,1]) ,random)
 

plt.subplot(221)
plt.title("a)")
plt.plot(a)
plt.subplot(222)
plt.title("b)")
plt.plot(b)
plt.subplot(223)
plt.title("c)")
plt.plot(c1)
plt.subplot(224)
plt.title("c)")
plt.plot(c2)
plt.show()

# Exercise 2 - Discrete Fourier Transformation - 4 Points
Implement the Discrete Fourier Transformation with [Matrix Multiplication](https://en.wikipedia.org/wiki/DFT_matrix).

In [ ]:
def dft_matrix(n):
    """
    Returns the Discrete Fourier Transformation Matrix of order `n`.
    """
    w  =  e ** ((-2j *pi)/n)
    matrix = (1/n**(1/2)) * np.fromfunction(lambda i,k: (w **(i*k)), (n ,n))
    return matrix

plt.subplot(121)
plt.imshow( abs( dft_matrix(8) ))
plt.subplot(122)
plt.imshow( abs(dft_matrix(32)))
 
plt.show()

# Exercise 3 - Spectrogram - 4 Points

A [spectrogram](https://en.wikipedia.org/wiki/Spectrogram) visualises the intensities of the different frequencies over time. On the x-axis is the time and on the y-axis are the frequencies.
Your task is to build the spectrogram of the given audio clip. If you failed to do exercise 2, you can use some library for the fourier transformation.

In [ ]:
def load_wav(url):
    f = urllib2.urlopen(url)
    buffer = BytesIO(f.read())
    sample_rate, audio = scipy.io.wavfile.read(buffer)
    if len(audio.shape) == 2:
        # only select one channel
        audio = audio[:, 0]
    return sample_rate, audio / audio.max()

In [ ]:
dolphines_url = "http://www.pmel.noaa.gov/acoustics/whales/sounds/whalewav/akhumphi1x.wav"
# some dolphin sounds
sample_rate, audio = load_wav(dolphines_url)

In [ ]:
print(audio.shape)

In [ ]:
# listen to the audio
adt.Audio(data=audio, rate=sample_rate)

In [ ]:
# plot the values.
plt.plot(audio)
plt.show()

In [ ]:
def spectogram(audio, window_size=500):
    
    """
    Builds the spectogram of the audio clip. Split the audio clip into buckets of `window_size` and 
    use the discrete fourier transformation to get the intensity of the frequencies.
    The entry at position (i, j) of the matrix is the intensity of the i-th frequencies in the j-th bucket.
    """
    # ignore the last 6 elements to split the signal in window_size=500 
    audio = audio[:-6]

    window_counter = audio.shape[0] // window_size

    # split the signal in buckets
    windows = np.array(np.split(audio,window_counter))

    # create the dft matrix
    matrix = dft_matrix(window_size)

    # init the result
    result = np.zeros((window_counter, int(window_size)))

    # calculate the dft for every window. 
    for i,window in enumerate(windows):
         result[i] = matrix @ window 

    return  result.T[250:]

In [ ]:
specgram = spectogram(audio) 

In [ ]:
im = plt.imshow(specgram)
plt.colorbar(im, fraction=0.02, pad=0.04)
plt.show()

In [ ]:
# plot it logarithmically
im = plt.imshow(specgram)    # your code here
plt.colorbar(im, fraction=0.02, pad=0.04)
plt.show()